## 런타임 유형 GPU로 바꾸고 시작하기!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/kubig/nlp/transcript_data_.csv", encoding='latin_1')

<ipython-input-6-09c954e91848>:1: DtypeWarning: Columns (244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/kubig/nlp/transcript_data_.csv", encoding='latin_1')


In [ ]:
#df['Unnamed: 2'].isnull().sum()

In [ ]:
df = df[df['Unnamed: 2'].isnull()]
df = df[df['transcript'].isnull()==False]
df = df.iloc[:,:2]
df.reset_index(drop=True,inplace=True)

In [ ]:
df.head()

,title,transcript
0,Can you outsmart the apples and oranges fallacy?,Baking apple pie? Discount orange warehouse ha...
1,The exploitation of US college athletes,"In college sports, American universities are e..."
2,How does ultrasound work?,"In a pitch-black cave, bats can?t see much. B..."
3,"An honest history of an ancient and ""nasty"" word","First, a warning. As far as offensive words go..."
4,The electrical blueprints that orchestrate life,"Chris Anderson: Mike, welcome. It's good to se..."


In [ ]:
def preprocess(df):
  list_=[]
  for i in range(len(df)):
    text = df['transcript'][i]
    letters_only = re.sub('[^a-zA-Z`,.]', ' ', text)
    lower_letters = letters_only.lower()
    spacing = re.sub(' +', ' ', lower_letters)
    list_.append(spacing)
  df['preprocessed']=list_
  return df

In [ ]:
df = preprocess(df)

In [ ]:
df = df.drop(["transcript"],axis=1)

In [ ]:
df.to_csv("preprocessed.csv")

In [ ]:
import csv
import json

def csv_to_jsonl(csv_file, jsonl_file):
    with open(csv_file, 'r', encoding='utf-8') as csv_in, open(jsonl_file, 'w', encoding='utf-8') as jsonl_out:
        reader = csv.DictReader(csv_in)
        for row in reader:
            jsonl_out.write(json.dumps(row, ensure_ascii=False) + '\n')

# CSV 파일 경로와 이름
csv_file = 'preprocessed.csv'

# JSONL 파일 경로와 이름
jsonl_file = 'output.jsonl'

# CSV 파일을 JSONL 파일로 변환
csv_to_jsonl(csv_file, jsonl_file)


In [ ]:
df_json = pd.read_json('preprocessed.jsonl')

ValueError: ignored

In [ ]:
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.3 MB/s eta 0:00:00


gpt 영어 모델 괜찮은 거 hugging face에서 찾기


In [ ]:
# Tokenizer 설정
from transformers import GPT2Tokenizer, GPT2TokenizerFast, GPT2Model

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2',
                                          bos_token='<|startoftext|>',
                                          eos_token='<|endoftext|>',
                                          pad_token='<|pad|>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print("본 모델의 최대 길이는 {} 입니다.".format(tokenizer.model_max_length))
print("Sequence token {}의 처음은 {} id를 갖고 있습니다.".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("Sequence token {}의 끝은 {} id를 갖고 있습니다.".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("Padding token {}은 {} id를 갖고 있습니다.".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

본 모델의 최대 길이는 1024 입니다.
Sequence token <|startoftext|>의 처음은 50257 id를 갖고 있습니다.
Sequence token <|endoftext|>의 끝은 50256 id를 갖고 있습니다.
Padding token <|pad|>은 50258 id를 갖고 있습니다.


In [ ]:
# 예시
example = df['preprocessed'][0]
print(example)

print(tokenizer.encode('<|startoftext|>' + example + '<|endoftext|>',
      truncation=True, max_length=768, padding="max_length")[1:])

baking apple pie discount orange warehouse has you covered a fruit s a fruit, right it s , and scientist james hansen has just testified to the united states congress that global warming trends are caused by human activity, and will pose an increasing threat to humanity in the future.well, well. that s unusually prescient for a human.looking for a wedding dress try a new take on a timeless classic. it s sleek, flattering and modest just like the traditional dress.commercials. could anything be more insufferable it s , and the united states senate has called a hearing about global warming. some expert witnesses point out that past periods in earth s history were warmer than the th century. because such variations existed long before humans, the witnesses claim the current warming trend is also the result of natural variation.ah, there is something more insufferable than a commercial. luckily for the humans, there s one more expert witness.what are you looking at we re all dressed. at le

In [ ]:
tok_ex = tokenizer(example)
print(tok_ex)
print(tok_ex.keys())

print(len(tok_ex['input_ids']))
print(len(tok_ex['attention_mask']))

{'input_ids': [65, 868, 17180, 2508, 9780, 10912, 20933, 468, 345, 5017, 257, 8234, 264, 257, 8234, 11, 826, 340, 264, 837, 290, 11444, 474, 1047, 289, 33807, 468, 655, 15463, 284, 262, 16503, 2585, 8681, 326, 3298, 9917, 11257, 389, 4073, 416, 1692, 3842, 11, 290, 481, 12705, 281, 3649, 2372, 284, 9265, 287, 262, 2003, 13, 4053, 11, 880, 13, 326, 264, 23708, 906, 3456, 329, 257, 1692, 13, 11534, 329, 257, 10614, 6576, 1949, 257, 649, 1011, 319, 257, 36464, 6833, 13, 340, 264, 33143, 11, 48259, 290, 12949, 655, 588, 262, 4569, 6576, 13, 36313, 82, 13, 714, 1997, 307, 517, 1035, 13712, 540, 340, 264, 837, 290, 262, 16503, 2585, 34548, 468, 1444, 257, 4854, 546, 3298, 9917, 13, 617, 5887, 11432, 966, 503, 326, 1613, 9574, 287, 4534, 264, 2106, 547, 23254, 621, 262, 294, 4289, 13, 780, 884, 13991, 11196, 890, 878, 5384, 11, 262, 11432, 1624, 262, 1459, 9917, 5182, 318, 635, 262, 1255, 286, 3288, 12291, 13, 993, 11, 612, 318, 1223, 517, 1035, 13712, 540, 621, 257, 5068, 13, 45120, 329, 262

In [ ]:
for i in tok_ex['input_ids']:
  print(tokenizer.convert_ids_to_tokens(i))

#출력된 거 보니까 G처럼 생긴 저 문자가 뭔진 모르겠는데 그냥 무시해도 되는듯!

b
aking
Ġapple
Ġpie
Ġdiscount
Ġorange
Ġwarehouse
Ġhas
Ġyou
Ġcovered
Ġa
Ġfruit
Ġs
Ġa
Ġfruit
,
Ġright
Ġit
Ġs
Ġ,
Ġand
Ġscientist
Ġj
ames
Ġh
ansen
Ġhas
Ġjust
Ġtestified
Ġto
Ġthe
Ġunited
Ġstates
Ġcongress
Ġthat
Ġglobal
Ġwarming
Ġtrends
Ġare
Ġcaused
Ġby
Ġhuman
Ġactivity
,
Ġand
Ġwill
Ġpose
Ġan
Ġincreasing
Ġthreat
Ġto
Ġhumanity
Ġin
Ġthe
Ġfuture
.
well
,
Ġwell
.
Ġthat
Ġs
Ġunusually
Ġpres
cient
Ġfor
Ġa
Ġhuman
.
looking
Ġfor
Ġa
Ġwedding
Ġdress
Ġtry
Ġa
Ġnew
Ġtake
Ġon
Ġa
Ġtimeless
Ġclassic
.
Ġit
Ġs
Ġsleek
,
Ġflattering
Ġand
Ġmodest
Ġjust
Ġlike
Ġthe
Ġtraditional
Ġdress
.
commercial
s
.
Ġcould
Ġanything
Ġbe
Ġmore
Ġins
uffer
able
Ġit
Ġs
Ġ,
Ġand
Ġthe
Ġunited
Ġstates
Ġsenate
Ġhas
Ġcalled
Ġa
Ġhearing
Ġabout
Ġglobal
Ġwarming
.
Ġsome
Ġexpert
Ġwitnesses
Ġpoint
Ġout
Ġthat
Ġpast
Ġperiods
Ġin
Ġearth
Ġs
Ġhistory
Ġwere
Ġwarmer
Ġthan
Ġthe
Ġth
Ġcentury
.
Ġbecause
Ġsuch
Ġvariations
Ġexisted
Ġlong
Ġbefore
Ġhumans
,
Ġthe
Ġwitnesses
Ġclaim
Ġthe
Ġcurrent
Ġwarming
Ġtrend
Ġis
Ġalso
Ġthe
Ġresult
Ġof
Ġnatural
Ġvariation
.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup

class GPT2Dataset(Dataset):
  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list: # script별로 encode한 뒤에 tensor로 만들어서 배열에 넣음
      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>',
                                 truncation=True, max_length=max_length,
                                 padding="max_length")
      self.input_ids.append(torch.tensor(encodings_dict['input_ids'][1:])) # CLS 토큰 제거
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask'][1:])) # CLS 토큰 제거

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]


In [ ]:
coverletter_list = df['preprocessed']
coverletter_list

0       baking apple pie discount orange warehouse has...
1       in college sports, american universities are e...
2       in a pitch black cave, bats can t see much. bu...
3       first, a warning. as far as offensive words go...
4       chris anderson mike, welcome. it s good to see...
                              ...                        
4211    about years ago, i took on the task to teach g...
4212    good morning. how are you audience good.it s b...
4213    if you re here today and i m very happy that y...
4214     music the sound of silence, simon amp garfunk...
4215    thank you so much, chris. and it s truly a gre...
Name: preprocessed, Length: 4216, dtype: object

In [ ]:
# Pytorch 데이터셋 생성
dataset = GPT2Dataset(coverletter_list, tokenizer, max_length=768)

# 훈련/검증 데이터셋으로 분리
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

3,794 training samples
  422 validation samples


In [ ]:
# Pytorch 데이터 로더
batch_size = 2 # GPT모델은 큰 모델이므로 batch_size를 2 초과시 메모리 용량 문제 발생

# Dataloaders for 훈련/검증 데이터셋
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # batch 랜덤으로 선택
            batch_size = batch_size # 선택된 batch_size로 훈련
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size # 선택된 batch_size로 검증
        )

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

In [ ]:
import random

model.resize_token_embeddings(len(tokenizer)) # token 사이즈 조정

seed_val = 425
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
# 하이퍼파라미터 튜닝
epochs = 3
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 1000

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                  )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# training steps = (number of batches) * (number of epochs)
total_steps = len(train_dataloader) * epochs

# learning rate scheduler (training loop이 지날수록 learning rate 달라짐)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps
                                            )

In [ ]:
import time
import datetime

def format_time(elapsed):
  return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
total_t0 = time.time()
training_stats = []
model = model.to(device)

# 1) Training
for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training Start')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device) # GPU 입력으로 사용될 tensor는 모두 to(device) 필요
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()      # 변화도(Gradient) 매개변수를 0으로 만들고

        outputs = model(b_input_ids, # loss를 출력하는지 확인
                        labels=b_labels,
                        attention_mask = b_masks,
                        token_type_ids=None
                        )

        loss = outputs[0]
        batch_loss = loss.item()
        total_train_loss += batch_loss

        # x batch마다 샘플 불러오기
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),  # beginning sequence token의 id (랜덤한 단어로 시작하계끔)
                                    do_sample=True,  # sampling 사용할지 여부 판단 (use greedy decoding)
                                    top_k=50, # number of highest probability vocab tokens
                                    max_length = 200, # maximum length of the sequence to be generated
                                    top_p=0.95,
                                    num_return_sequences=1, # 한 배치당 returned sequence의 수
                                    repetition_penalty=2.0,
                                    )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train() # train은 일정 sample_every step마다

        loss.backward()
        optimizer.step()
        scheduler.step()

    # 모든 batch에 대해 평균 loss 계산
    avg_train_loss = total_train_loss / len(train_dataloader)

    # epoch별로 소요시간 측정
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    # 2) Validation
    print("")
    print("Validation Start")

    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
                            #token_type_ids=None,
                            attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training Complete")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training Start


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,000  of  1,897. Loss: 3.1972579956054688.   Elapsed: 0:08:17.
0: mega of all things we ve learned in life, how much we care about what is best for the future. so let s talk to you today and just consider it a very simple question why would i spend my time here if there were only seconds left by tomorrow morning at midnight laughter well...today everybody does some math on that day when they re most tired or asleep hours long every single hour. million billion dollars equals this energy saving project right now, trillion times better than before because electricity has been released into our atmosphere over years like steam powered cars do tonight instead gas was created less four minutes ago but since then millions people have died from breathing carbon dioxide around us using air pollution as their source transportation fuel while everyone else will die soon after with respiratory illness including your neighbor who still doesn t know she needs help without her breath smelli

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,000  of  1,897. Loss: 3.180615186691284.   Elapsed: 0:08:27.
0: de, we have seen a rise of social inequalities, inequality and the political will to tackle this economic crisis. i want you all here today for an open discussion about one aspect that s more significant than other problems in life what are our weaknesses as human beings if they don t work out how badly then everything else falls apart from poverty is bad enough it goes up against capitalism because people do not need money or education yet but instead rely on government welfare dollars these systems make wealth irrelevant at every turn whether there be good public goods like water filters running electricity supply through walls so poor infrastructure deteriorates over time until justifications come along with programs designed less costly by hand management techniques look something similar when implemented properly others reallocate resources among those who already exist which leads us astray into disaster mo

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 1,000  of  1,897. Loss: 2.739055633544922.   Elapsed: 0:08:27.
0: fficient to build robots by taking advantage of the human capabilities for automation, researchers at harvard and yale have created a new toolkit which takes this concept from artificial intelligence that does amazing things but also uses it with some really neat tricks.take these two smart machines known as lidar bots or lindradotokas we ll call them lasershikumsara taku in china gaijin robotics company has developed an autonomously controlling robot using light emitting diodes called lasers specifically tuned around its laser target so they re sensitive enough not only on detecting objects within range, near infrared radiation no other technology can do just one thing very quickly destroy something without harming anyone else s lives firstly burn parts off if there is damage second most importantly freeze dead body parts so instead their core would be buried away in hot coal rods third use microwaves powered th

In [ ]:
params = list(model.named_parameters())
print('GPT-2 모델은 {:} 개의 다른 파라미터를 갖습니다.\n'.format(len(params)))

print('==== Embedding Layer ====\n')
for p in params[0:2]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')
for p in params[2:14]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')
for p in params[-2:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

GPT-2 모델은 148 개의 다른 파라미터를 갖습니다.

==== Embedding Layer ====

transformer.wte.weight                                  (50259, 768)
transformer.wpe.weight                                   (1024, 768)

==== First Transformer ====

transformer.h.0.ln_1.weight                                   (768,)
transformer.h.0.ln_1.bias                                     (768,)
transformer.h.0.attn.c_attn.weight                       (768, 2304)
transformer.h.0.attn.c_attn.bias                             (2304,)
transformer.h.0.attn.c_proj.weight                        (768, 768)
transformer.h.0.attn.c_proj.bias                              (768,)
transformer.h.0.ln_2.weight                                   (768,)
transformer.h.0.ln_2.bias                                     (768,)
transformer.h.0.mlp.c_fc.weight                          (768, 3072)
transformer.h.0.mlp.c_fc.bias                                (3072,)
transformer.h.0.mlp.c_proj.weight                        (3072, 768)
transformer.h

In [ ]:
output_dir = '/content/drive/MyDrive/model_save'

# 디렉토리 만들기
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print("Saving model to %s" % output_dir)

# 모델, 토크나이저 등 저장
model_to_save = model.module if hasattr(model, 'module') else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/drive/MyDrive/model_save


('/content/drive/MyDrive/model_save/tokenizer_config.json',
 '/content/drive/MyDrive/model_save/special_tokens_map.json',
 '/content/drive/MyDrive/model_save/vocab.json',
 '/content/drive/MyDrive/model_save/merges.txt',
 '/content/drive/MyDrive/model_save/added_tokens.json',
 '/content/drive/MyDrive/model_save/tokenizer.json')

In [ ]:
!pip3 install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.3 MB/s eta 0:00:00


In [ ]:
# 저장한 모델 불러오기
from transformers import GPT2TokenizerFast, GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/model_save')
tokenizer = GPT2TokenizerFast.from_pretrained('/content/drive/MyDrive/model_save')

In [ ]:
import torch
import random

device = torch.device("cuda")
model.cuda()

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
model = model.to(device)

In [ ]:
model.eval()

def coverletter_generator(text):
  prompt = f"<|startoftext|> {text}"
  generated = torch.tensor(tokenizer.encode(prompt)[1:]).unsqueeze(0)
  generated = generated.to(device)

  print(generated)

  sample_outputs = model.generate(
                                  generated,
                                  do_sample=True,
                                  top_k=50,
                                  max_length = 300,
                                  top_p=0.95,
                                  num_return_sequences=3,
                                  repetition_penalty=1.1
                                  )

  for i, sample_output in enumerate(sample_outputs):
    result = "{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True))
    print(result)
  return

In [ ]:
#원하는 텍스트 입력!
answer_generator("In order to get good grades in college, you have to study hard first.")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[  554,  1502,   284,   651,   922, 19051,   287,  4152,    11,   345,
           423,   284,  2050,  1327,   717,    13]], device='cuda:0')
0:  In order to get good grades in college, you have to study hard first. this requires a lot of perseverance, and that is why i m here today on the ted stage so everyone can participate in it. and this is not about being smart or funny. for one thing people are much smarter than we think they should be, but this isn t just about being smart. it s also about doing things you want to do yourself.so how do we measure our intelligence let us start with taking a test, like i did three years ago at the university of texas, where two young people put together this puzzle using an app called amazon machine learning systems. amazon machine learning systems can make these predictions on social network sites, by looking at content shared between user specific images that look different than average ones from real life photos. you can see from what s